In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from category_encoders import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer

from xgboost import XGBRegressor

# Data Cleaning

In [2]:
# # unzip files
# !unzip Other/Inpatient_Data_2011_CSV.zip
# !unzip Other/Inpatient_Data_2012_CSV.zip
# !unzip Other/Inpatient_Data_2013_CSV.zip
# !unzip Other/Inpatient_Data_2014_CSV.zip
# !unzip Other/Inpatient_Data_2015_CSV.zip
# !unzip Other/Inpatient_Data_2016_CSV.zip

In [3]:
# load data into pandas
top_2011 = pd.read_csv('Data/Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv')
top_2012 = pd.read_csv('Data/Medicare_Provider_Charge_Inpatient_DRG100_FY2012.csv')
top_2013 = pd.read_csv('Data/Medicare_Provider_Charge_Inpatient_DRG100_FY2013.csv')
og_2014 = pd.read_csv('Data/Medicare_Provider_Charge_Inpatient_DRGALL_FY2014.csv')
og_2015 = pd.read_csv('Data/Medicare_Provider_Charge_Inpatient_DRGALL_FY2015.csv')
og_2016 = pd.read_csv('Data/Medicare_Provider_Charge_Inpatient_DRGALL_FY2016.csv')

/home/eric/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# define top100 list
top100 = top_2011['DRG Definition'].unique().tolist()

In [5]:
# get diagnoses from top 100
top_2014 = og_2014[og_2014['DRG Definition'].isin(top100)]
top_2015 = og_2015[og_2015['DRG Definition'].isin(top100)]
top_2016 = og_2016[og_2016['DRG Definition'].isin(top100)]

In [6]:
# add year feature
top_2011['year'] = [2011] * top_2011.shape[0]
top_2012['year'] = [2012] * top_2012.shape[0]
top_2013['year'] = [2013] * top_2013.shape[0]
top_2014['year'] = [2014] * top_2014.shape[0]
top_2015['year'] = [2015] * top_2015.shape[0]
top_2016['year'] = [2016] * top_2016.shape[0]

/home/eric/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/eric/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/eric/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [7]:
# 2016 has a '$' in front of `Average Total Payments`
def strip_dollar(entry):
    return float((entry.strip('$')).replace(',', ''))

top_2016['Average Total Payments'] = top_2016['Average Total Payments'].apply(strip_dollar)
top_2016['Average Medicare Payments'] = top_2016['Average Medicare Payments'].apply(strip_dollar)

/home/eric/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/eric/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
# concatenate dataframes
dataframes = [top_2011, top_2012, top_2013, top_2014, top_2015, top_2016]
top100 = pd.concat(dataframes)

In [9]:
# change name
top100['diagnosis'] = top100['DRG Definition']
top100 = top100.drop(columns='DRG Definition')

In [10]:
# drop columns
top100_clean = top100.drop(columns=['Provider Id',
                     'Provider Name',
                     'Provider Street Address',
                     'Provider City',
                     'Provider Zip Code',
                     'Hospital Referral Region (HRR) Description',
                     'Total Discharges',
                     'Average Covered Charges'])

In [11]:
# create out-of-pocket payment feature
top100_clean['cost'] = top100_clean['Average Total Payments'] - top100_clean['Average Medicare Payments']

# now drop other payment features
top100 = top100_clean.drop(columns=['Average Total Payments',
                                    'Average Medicare Payments'])

In [12]:
# get rid of beginning
def strip_beginning(entry):
    return str(entry)[6:]

top100['diagnosis'] = top100['diagnosis'].apply(strip_beginning)

In [13]:
# get rid of endings
endings = [' W/O CC/MCC',
           ' W/O MCC',
           ' W MCC',
           ' W CC',
           ' W/O CC',
           ' W CC/MCC']

def strip_endings(entry):
    for ending in endings:
        if entry.endswith(ending):
            return entry.replace(ending, '')
            
top100['diagnosis'] = top100['diagnosis'].apply(strip_endings)

In [14]:
# categorize diagnoses
diagnoses = top100['diagnosis'].unique().tolist()

# neuro
neuro = diagnoses[:3] + diagnoses[4:6]

# respiratory
respiratory = diagnoses[6:11]

# circulatory
circulatory = diagnoses[11:25]

# digestive
digestive = diagnoses[25:33]

# orthopedic
orthopedic = diagnoses[33:41] + diagnoses[53:]

# PCP
pcp = diagnoses[41:44]

# nephrology
nephrology = diagnoses[44:47]

# other
other = diagnoses[47:53] + diagnoses[3:4]

In [15]:
def categorize(diagnosis):
    if diagnosis in neuro:
        return 'Neurological'
    elif diagnosis in respiratory:
        return 'Respiratory'
    elif diagnosis in circulatory:
        return 'Circulatory'
    elif diagnosis in digestive:
        return 'Digestive'
    elif diagnosis in orthopedic:
        return 'Orthopedic'
    elif diagnosis in pcp:
        return 'PCP'
    elif diagnosis in nephrology:
        return 'Nephrology'
    elif diagnosis in other:
        return 'Other'
    else:
        return 'Other'

In [16]:
# categorize diagnoses
# top100['diagnosis'] = top100['diagnosis'].apply(categorize)

# Encode & Impute

In [17]:
# instantiate encoder
encoder = OrdinalEncoder()

In [18]:
encoded = encoder.fit_transform(top100)

In [19]:
# instantiate imputer
imputer = SimpleImputer(np.nan, strategy='median')

In [20]:
imputed = pd.DataFrame(imputer.fit_transform(encoded), columns = encoded.columns.values)

In [21]:
imputed.head()

,Provider State,year,diagnosis,cost
0,1.0,2011.0,1.0,1013.505494
1,1.0,2011.0,1.0,810.857143
2,1.0,2011.0,1.0,981.166666
3,1.0,2011.0,1.0,1288.400000
4,1.0,2011.0,1.0,806.888889


In [22]:
# replace zeroes with ones
def zero_to_one(entry):
    if entry == 0:
        return 1
    else:
        return entry

In [23]:
# replace
imputed['cost'] = imputed['cost'].apply(zero_to_one)

# Train/Test Split

In [24]:
# train/test split
train = pd.concat([imputed[imputed['year'] == 2011],
           imputed[imputed['year'] == 2012],
           imputed[imputed['year'] == 2013],
           imputed[imputed['year'] == 2014],
           imputed[imputed['year'] == 2015]])

test = imputed[imputed['year'] == 2016]

In [25]:
# define target and features
target = 'cost'
features = ['diagnosis', 'Provider State']

In [26]:
# X matrices
X_train = train[features]
X_test = test[features]

# y vector
y_train = train[target]
y_test = test[target]

# Baseline Model

In [27]:
# define error metric
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [38]:
# find median cost
median = train['cost'].median()

# predictions
y_pred = [median] * len(test)

# accuracy
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", rmse(y_test, y_pred))

MAE: 698.7045627582952
RMSE: 1546.465099275328


# RF

In [39]:
# instantiate model
rf = RandomForestRegressor()

# fit model
rf.fit(X_train, y_train)

# predict
y_pred = rf.predict(X_test)

# accuracy
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", rmse(y_test, y_pred))

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MAE: 636.6854783231073
RMSE: 1335.4262660109007


In [30]:
## Log transformed
y_log_train = np.log(y_train)
y_log_test = np.log(y_test)

In [40]:
# instantiate model
rf = RandomForestRegressor()

# fit model
rf.fit(X_train, y_log_train)

# predict
y_pred = np.exp(rf.predict(X_test))

# accuracy
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", rmse(y_test, y_pred))

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MAE: 624.5603410846749
RMSE: 1393.20395437988


In [41]:
# instantiate model
rf = RandomForestRegressor()

# fit model
rf.fit(X_train, y_train)

# predict
y_pred = rf.predict(X_test)

# accuracy
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", rmse(y_test, y_pred))

/home/eric/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MAE: 636.7962652421903
RMSE: 1335.472441039065


# XGBoost Regressor

In [42]:
# Instantiate model
rf = XGBRegressor()

# fit model
rf.fit(X_train, y_train)

# predict
y_pred = rf.predict(X_test)

# accuracy
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", rmse(y_test, y_pred))

/home/eric/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/eric/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[17:12:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE: 641.9442020303599
RMSE: 1346.138762531945


In [43]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

In [50]:
pipeline = make_pipeline(
    OrdinalEncoder(), 
    XGBRegressor(random_state=42)
)

param_distributions = {
    'xgbregressor__n_estimators': randint(100, 1000), 
    'xgbregressor__max_depth': randint(3, 50),
    'xgbregressor__eta': uniform(0.01, 0.3)
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=15, 
    cv=5, 
    scoring='neg_mean_squared_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed: 23.7min
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed: 31.0min
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed: 38.3min
[Parallel(n_jobs=-1)]: Done  72 out of  75 | elapsed: 42.4min remaining:  1.8min
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed: 43.1min finished


[18:18:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [51]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation RMSE', np.sqrt(-search.best_score_))

Best hyperparameters {'xgbregressor__eta': 0.2826213009198212, 'xgbregressor__max_depth': 7, 'xgbregressor__n_estimators': 515}
Cross-validation RMSE 1554.4537275632592


In [53]:
# Instantiate model
rf = XGBRegressor(n_estimators=515,
                  max_depth=7,
                  eta=0.2826213,
                  random_state=42,
                  n_jobs=-1)

# fit model
rf.fit(X_train, y_train)

# predict
y_pred = rf.predict(X_test)

# accuracy
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", rmse(y_test, y_pred))

[18:22:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE: 636.1796795292753
RMSE: 1334.5831947893294
